In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

In [2]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages

In [3]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph
from langchain_teddynote.tools.tavily import TavilySearch

In [4]:
from langchain.schema import HumanMessage, AIMessage
from langchain_core.messages import ToolMessage

In [5]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [6]:
llm = ChatOpenAI( 
    api_key=key, 
    model_name='gpt-4o-mini',
    temperature=0.1
)

In [7]:
tool = TavilySearch(max_results=1)      # 검색 도구 생성
tools = [tool]                          # 도구 목록에 넣기  

In [8]:
# LLM 에 도구 바인딩
llm_with_tools = llm.bind_tools(tools)

In [9]:
def chatbot(state: State):
    print('===== chatbot() 함수 시작 =====')
    
    print("[1] chatbot() 으로 넘어온 메시지: ")

    message_type1 = ''

    for msg in state['messages']:
        if isinstance(msg, HumanMessage):
            message_type1 = message_type1 + '[HumanMessage]'
        elif isinstance(msg, AIMessage):
            message_type1 = message_type1 + '[AIMessage]'
        elif isinstance(msg, ToolMessage):
            message_type1 = message_type1 + '[ToolMessage]'
        
        print(f'메시지 타입: {message_type1}')
        print(msg)
        print()

    print(f"\n[2] 메시지 개수 : {len(state['messages'])}\n")

    answer = llm_with_tools.invoke(state['messages'])

    # print(f'[도구 사용 LLM 실행 결과 content]: {answer.content}')
    
    print('[3] chatbot()에서 실행:')
    print('메시지 타입: ', end='')

    message_type2 = ''
    if isinstance(answer, AIMessage):
        message_type2 = message_type2 + '[AIMessage]'
    elif isinstance(answer, HumanMessage):
        message_type2 = message_type2 + '[HumanMessage]'
    elif isinstance(answer, ToolMessage):
        message_type2 = message_type2 + '[ToolMessage]'
    else:
        message_type2 = type(answer)

    print(message_type2)
    print(answer)
    print()

    answer_value = {'messages': [answer]}

    print(f"[4] chatbot()에서 실행 후 메시지 개수: {message_type1} {message_type2} {len(state['messages']) + len(answer_value)}") 
    print('===== chatbot() 함수  끝 =====')
    print()

    return answer_value

In [10]:
question = "대구의 유명한 맛집을 알려줘."

state1 = State(messages=[('user', question)])
answer_state = chatbot(state1)

===== chatbot() 함수 시작 =====
[1] chatbot() 으로 넘어온 메시지: 
메시지 타입: 
('user', '대구의 유명한 맛집을 알려줘.')


[2] 메시지 개수 : 1

[3] chatbot()에서 실행:
메시지 타입: [AIMessage]
content='' additional_kwargs={'tool_calls': [{'id': 'call_8wKvL8K6l6u9pwfZBP92HDcP', 'function': {'arguments': '{"query":"대구 유명 맛집"}', 'name': 'tavily_web_search'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 102, 'total_tokens': 124, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b8bc95a0ac', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-21ec95be-7a4d-491d-a2e2-e41fa420f4b5-0' tool_calls=[{'name': 'tavily_web_search', 'args': {'query': '대구 유명 맛집'}, 'id': 'call_8wKvL8K6l6u9pwfZBP92HDcP', 'type': 'tool_call'}] usage_metadata={'input_tok

In [11]:
answer_state

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8wKvL8K6l6u9pwfZBP92HDcP', 'function': {'arguments': '{"query":"대구 유명 맛집"}', 'name': 'tavily_web_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 102, 'total_tokens': 124, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_b8bc95a0ac', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-21ec95be-7a4d-491d-a2e2-e41fa420f4b5-0', tool_calls=[{'name': 'tavily_web_search', 'args': {'query': '대구 유명 맛집'}, 'id': 'call_8wKvL8K6l6u9pwfZBP92HDcP', 'type': 'tool_call'}], usage_metadata={'input_tokens': 102, 'output_tokens': 22, 'total_tokens': 124, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_d

In [14]:
from langgraph.graph import StateGraph, START, END
import json

def route_tools(state: State):

    if messages := state.get('messages', []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    
    print()
    print('===== 라우터 노드 =====')
    # print(f"라우터 노드로 넘어온 chatbot() 에서 생성한 state 메시지: ")
    # print(ai_message)

    if hasattr(ai_message, 'tool_calls') and len(ai_message.tool_calls) > 0:
        print(f'라우터 [tools] : \n{ai_message.tool_calls}')
        print(f'라우터 리턴: tools')
        print('===== 라우터 노드 끝 =====')
        print()

        return 'tools'
    
    print(f'라우터 리턴: {END}')
    print('===== 라우터 노드 끝 =====')
    print()
    
    return END

In [15]:
state_with_tools = {
    "messages": [
        HumanMessage(
            content="대한민국 수도에 대해서 검색해줘",
            id='6e0ff6a0-d790-42a0-ae24-28172c7feabc'
        ),
        AIMessage(
            content="",
            tool_calls=[{
                "name": "tavily_web_search",                        
                "args": {"query": "대한민국 수도"},
                "id": "call_k6ljrhTRP8dz8FwNrtbyyG3K"
            }],
            additional_kwargs={
                "tool_calls": [{
                    "id": "call_k6ljrhTRP8dz8FwNrtbyyG3K",
                    "function": {
                        "arguments": json.dumps({"query": "대한민국 수도"}),
                        "name": "tavily_web_search"                     
                    },
                    "type": "function"
                }],
                "refusal": None
            },
            response_metadata={
                "token_usage": {
                    "completion_tokens": 20,
                    "prompt_tokens": 99,
                    "total_tokens": 119
                },
                "model_name": "gpt-4o-mini-2024-07-18"
            },
            id='run-164b42b2-a16c-4ea8-8a2e-94da55991426-0'
        )
    ]
}


route_tools(state_with_tools)


===== 라우터 노드 =====
라우터 [tools] : 
[{'name': 'tavily_web_search', 'args': {'query': '대한민국 수도'}, 'id': 'call_k6ljrhTRP8dz8FwNrtbyyG3K', 'type': 'tool_call'}]
라우터 리턴: tools
===== 라우터 노드 끝 =====



'tools'

In [16]:
import json
from langchain_core.messages import ToolMessage

class BasicToolNode:
    """Run tools requested in the last AIMessage node"""

    def __init__(self, tools: list) -> None:
        self.tools_list = {tool.name: tool for tool in tools}
        print(f'======================================================')
        print('[BasicToolNode]')
        print('도구 호출 생성자')
        print(f'tools_list: {self.tools_list}')
        print(f'======================================================')
    
    def __call__(self, inputs: dict):
        if messages := inputs.get('messages', []):
            message = messages[-1]
        else:
            raise ValueError('No message found in input')
        
        print()
        print(f'======================================================')
        print('[BasicToolNode] call')
        #print('도구 호출로 갔을 때')
        #print('message.tool_calls:', message.tool_calls)
        #print(f'======================================================')

        outputs = []

        for tool_call in message.tool_calls:    # message의 tool_calls 속성이 있는것은 도구 호출을 필요한 메시지가 있는 경우
            print(f'도구 호출이 필요한 경우: ')
            print(f'도구 호출 : {tool_call}')
            print(f"도구 호출 이름: {tool_call['name']}")
            print(f"도구 호출 인자 : {tool_call['args']}")

            tool_result = self.tools_list[tool_call['name']].invoke(tool_call['args'])
            # print(f'도구 호출 결과 : {tool_result}')                                           
        
            outputs.append(
                ToolMessage(
                    content=json.dumps(tool_result, ensure_ascii=False), 
                    name=tool_call['name'], 
                    tool_call_id=tool_call['id']
                )
            )
            
        print('[BasicToolNode] call 끝')
        print(f'======================================================')
        print()

        return {'messages': outputs}    

In [17]:
# 도구 노드 생성
tool_node = BasicToolNode(tools=[tool])   

[BasicToolNode]
도구 호출 생성자
tools_list: {'tavily_web_search': TavilySearch(client=<tavily.tavily.TavilyClient object at 0x000001A17F90FFD0>, max_results=1)}


In [18]:
from langchain_core.runnables import RunnableLambda

tool_node_runnable = RunnableLambda(tool_node)

In [19]:
sample_input = State(
    messages=[
        HumanMessage(content='대구 맛집을 알려줘'),
        AIMessage(
            content='',
            tool_calls=[{
                    'name': 'tavily_web_search', 
                    'args': {'query': '대구 맛집'},
                    'id': '99c32be8-e500-4b2c-9675-34ab45fa6ab5'
                }]
        )
    ]
)

result = tool_node(sample_input)

for msg in result["messages"]:
        print(f"- {msg.name}: {msg.content}")


[BasicToolNode] call
도구 호출이 필요한 경우: 
도구 호출 : {'name': 'tavily_web_search', 'args': {'query': '대구 맛집'}, 'id': '99c32be8-e500-4b2c-9675-34ab45fa6ab5', 'type': 'tool_call'}
도구 호출 이름: tavily_web_search
도구 호출 인자 : {'query': '대구 맛집'}
[BasicToolNode] call 끝

- tavily_web_search: [{"title": "대구 맛집 베스트10 유명해서 많이 방문하는 곳 Top10", "url": "https://todaytrip.tistory.com/141", "content": "대구 맛집 베스트10은 대덕식당, 오퐁드부아, 바르미스시뷔페 두산점 등 유명한 음식점부터 안지랑 곱창골목까지 다양한 분위기의 곳을 소개합니다. 각 곳의 주소, 영업시간, 메뉴, TV방송정보, 홈페이지 등을 확인하고 대구 맛집을 찾아보세요.", "score": 0.8565368, "raw_content": "일상탈출\n\n고정 헤더 영역\n\n\n\n메뉴 레이어\n\n메뉴 리스트\n\n검색 레이어\n\n검색 영역\n\n상세 컨텐츠\n\n본문 제목\n\n대구 맛집 베스트10 유명해서 많이 방문하는 곳 TOP10\n\nby 일탈스토리\n2023. 2. 24. 03:13\n\n본문\n\n안녕하세요. 전국맛집을 소개시켜드리는 일상탈출입니다.  오늘은 대구 맛집 추천해 드리겠습니다. 대구 맛집 베스트10은 사람들이 많이 방문하는곳이며, 웨이팅 시간이 있는 음식점도 있습니다. 대구 맛집 베스트10은 대덕식당, 오퐁드부아, 바르미스시뷔페 두산점, 헤이마, 곤지곤지, 서민갈비, 고령촌돼지찌개, 룰리커피 가창점, 리안, 낙영찜갈비 본점  10곳과 특별히 안지랑 곱창골목까지 총 11곳 알려드리겠습니다.\n\n\n\n1. 대덕식당\n\nㅇ 주소 : 대구 남구 앞산순환로 443 ㅇ 영업시간 : 07:00 - 21:00ㅇ 